In [ ]:
class Process:
    def __init__(self, name, priority, time_slice, power_consumption):
        self.name = name
        self.priority = priority
        self.time_slice = time_slice
        self.power_consumption = power_consumption
        self.remaining_time = time_slice
        self.start_time = 0
        self.power_used = 0
        
    def __str__(self):
        return f'{self.name} (priority: {self.priority}, time slice: {self.time_slice}, power consumption: {self.power_consumption})'
        
class CFS:
    def __init__(self, processes):
        self.processes = processes
        self.total_power_used = 0
        
    def run(self):
        current_time = 0
        
        while self.processes:
            # Sort the processes by priority and time slice
            self.processes.sort(key=lambda p: (p.priority, p.remaining_time))
            
            # Get the highest priority process
            current_process = self.processes[0]
            
            # Calculate the power used by the current process
            if current_process == self.processes[0]:
                power_used = current_process.power_consumption + 1
            else:
                power_used = current_process.power_consumption + 0.5
            
            # Update power used by the system
            self.total_power_used += power_used
            
            # Update the start time of the current process
            if current_process.remaining_time == current_process.time_slice:
                current_process.start_time = current_time
                
            # Execute the current process for one time slice
            current_process.remaining_time -= 1
            
            # Remove the process if it's finished
            if current_process.remaining_time == 0:
                print(f'{current_process.name} finished at time {current_time}')
                self.processes.remove(current_process)
                
            # Update the current time
            current_time += 1
        
        print(f'Total power used by the system: {self.total_power_used}W')